In [ ]:
# pip install -r requirements.txt

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

# Define model ID and local paths
model_id = "microsoft/Phi-4-mini-instruct"
model_folder = "LLMs/phi-4-mini-instruct"
tokenizer_folder = "Tokenizers/phi-4-mini-instruct"

# Function to check if directory is already populated
def is_downloaded(directory):
    return os.path.exists(directory) and any(os.scandir(directory))

# Create folders if necessary
os.makedirs(model_folder, exist_ok=True)
os.makedirs(tokenizer_folder, exist_ok=True)

# Download tokenizer if not already present
if not is_downloaded(tokenizer_folder):
    print("⬇️ Downloading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    tokenizer.save_pretrained(tokenizer_folder)
    print(f"✅ Tokenizer saved to: {tokenizer_folder}")
else:
    print(f"✅ Tokenizer already exists at: {tokenizer_folder}")

# Download model if not already present
if not is_downloaded(model_folder):
    print("⬇️ Downloading model...")
    model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
    model.save_pretrained(model_folder)
    print(f"✅ Model saved to: {model_folder}")
else:
    print(f"✅ Model already exists at: {model_folder}")


✅ Tokenizer already exists at: Tokenizers/phi-4-mini-instruct
✅ Model already exists at: LLMs/phi-4-mini-instruct


In [1]:
from src.backend.agents.agent import PersoAgent

In [2]:

agent = PersoAgent(
    model_path="LLMs/phi-4-mini-instruct",
    tokenizer_path="Tokenizers/phi-4-mini-instruct"
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


ValueError: ZeroShotAgent does not support multi-input tool User Profiler.

In [ ]:

response = agent.handle_task("I love drinking green tea in the afternoon.")
print(response)


In [ ]:
# Cell 2: Define Test Prompt
task = "I really enjoy watching action movies and drinking black coffee."

# Cell 3: Run Agent
response = agent.handle_task(task)
print("Agent Response:", response)

In [4]:
import google.protobuf
print("✅ Protobuf version:", google.protobuf.__version__)


✅ Protobuf version: 6.31.1


In [1]:
from langchain_community.llms import HuggingFacePipeline
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Load from local folders
tokenizer = AutoTokenizer.from_pretrained("Tokenizers/phi-4-mini-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "LLMs/phi-4-mini-instruct", 
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Simple calculator tool
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except:
        return "Invalid expression"

tools = [Tool(name="Calculator", func=calculator, description="Calculate math expressions")]

# Create agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Test
response = agent.run("What is 15 * 8?")
print(response)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipykernel_2119820/1866641229.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_2119820/1866641229.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <ht



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Calculator(expression: str) -> str - Calculate math expressions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is 15 * 8?
Thought: I need to multiply 15 by 8 to get the answer.
Action: Calculator
Action Input: 15 * 8
Observation: The result of 

In [ ]:
# pip install accelerate